# Traffic Flow with PDEs

#### <font color='gray'>Authors: Rob Hesselink(rob.hesselink@ens.fr) and Moshir Harsh (moshir.harsh@ens.fr)</font>

In this project model a traffic flow on a one dimensional road with periodic boundary conditions which is essentially a circular road.

We use the following equation called the WLR Equation:

$$ \frac{\partial \rho}{\partial t} + \frac{\partial}{\partial x}\left( v_0 \left(1-\frac{\rho}{\rho_0}\right)\rho \right) = 0 $$

where $v_0$ and $\rho_0$ are the parameters which are set to one without loss of generality. So the equation becomes:

$$ \frac{\partial \rho}{\partial t} + \frac{\partial}{\partial x}\left(\left(1-\rho\right)\rho \right) = 0 $$

This equation can also be written as:

$$ \frac{\partial \rho}{\partial t} + \left(1-2\rho\right)\frac{\partial \rho}{\partial x} = 0 $$

This equation is known to form shocks (traffic jams) and is like the Burger's equation with the velocity which depends on density instead of being constant.

We simulate the solution to this equation using different schemes and compare our results. We also discuss the dispersive and/or the disspipative nature of these schemes.

In [39]:
# UNIVERSAL (ALWAYS RUN FIRST)

#Importing Packages
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
#matplotlib inline
from matplotlib import animation, rc
from IPython.display import HTML

#Parameters:
N = 100.
dx = 1/N
dt = 1/(N)
T = 1.3
v0 = 1
rho0 = 1



#Initial Conditions: Here we define various initial conditions, only uncomment the one to be used.

#Sin Wave
p0 =  0.2 + 0.1*np.sin(np.arange(0, 2*np.pi, 2*np.pi*dx))
x = np.arange(0, 1, dx)

#Gaussian


#constant and non-constant:
#p0 = np.zeros(Nx)

# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim((0, 1))
ax.set_ylim((0, .5))
line, = ax.plot([], [], lw=2)
ax.set_ylabel("Rho")
ax.set_xlabel("x")





def init():
    line.set_data(x, p)
    time = ax.annotate('T = ' + str(0), xy=(.8, .45))
    mass = ax.annotate('Mass = ' + str(np.sum(p0)), xy=(.8, .4))
    time.set_animated(True)
    mass.set_animated(True)
    return line, time, mass

def animate(i, scheme):
    global p
    p = scheme(p)
    line.set_data(x, p)
    
    time.s = "T = " + str(i*dt)
    mass.s = "Mass = " + str(3)
    return line, time, mass
  
# Helper functions for conservative methods/Godunov method

def flux(shift, p):    
    """ Helper function for conservative schemes. """
    if shift == 0:
        return v0*(1 - p/rho0)*p
    else:
        return v0*(1 - np.roll(p, -shift)/rho0)*np.roll(p, -shift)

def velocity(shift = 0, array=p):
    """ Helper function for Godunov scheme. """
    if shift == 0:
        return v0*(1 - array/rho0)
    else:
        return v0*(1 - np.roll(array, -shift)/rho0)
    

## Finite Element Methods

### Lax-Friedrichs Finite Element Scheme

In [40]:
def LF(p):
    """ Finite elements Lax-Friedrichs """    
    A = (np.roll(p, -1) + np.roll(p, 1))/2.
    B = -dt*(1 - 2*p)*(np.roll(p, -1) - np.roll(p, 1))/(2*dx)
    return (A + B)


anim = animation.FuncAnimation(fig, animate, init_func=init, fargs=(LF,),
                               frames=int(T/dt), interval=50, blit=True)
p = np.copy(p0)
ax.set_title("Lax-Friedrichs")
rc('animation', html='html5')
HTML(anim.to_html5_video())
anim


### Lax-Wendroff Finite Element Scheme

In [18]:
def LW(p):
    """ Finite elements Lax-Wendroff """
    A = -(1 - 2*p)*(np.roll(p, -1) - np.roll(p, 1))/(2*dx)
    B = dt/2*(1 - 2*p)**2*(np.roll(p, -1) - 2*p + np.roll(p, 1))/(dx*dx)
    p += dt*(A + B)
    return p

anim = animation.FuncAnimation(fig, animate, init_func=init, fargs=(LW,),
                               frames=int(T/dt), interval=50, blit=True)
p = np.copy(p0)
ax.set_title("Lax-Wendroff")
rc('animation', html='html5')
HTML(anim.to_html5_video())
anim


##### Explaination:
Lax Wendroff is a dipersive scheme, we are loosing cars (actually gaining cars). Once the schock has formed, we should just get a propogation of the schock at a constant velocity.
But for Lax Freidrich, it is a dipersive scheme. We form a schock but we would observe that the shock starts to dissappear because the scheme is dispersive, the cars disperse and flattern the schock out. This should not happen.

## Finite Volume Methods

### Lax-Wendroff Finite Volumes

$$ u_{{i+1/2}}^{{n+1/2}}={\frac  {1}{2}}(u_{{i+1}}^{n}+u_{{i}}^{n})-{\frac  {\Delta t}{2\,\Delta x}}(f(u_{{i+1}}^{n})-f(u_{{i}}^{n})) $$

$$ u_{{i-1/2}}^{{n+1/2}}={\frac  {1}{2}}(u_{{i}}^{n}+u_{{i-1}}^{n})-{\frac  {\Delta t}{2\,\Delta x}}(f(u_{{i}}^{n})-f(u_{{i-1}}^{n})) $$

$$ u_{i}^{{n+1}}=u_{i}^{n}-{\frac  {\Delta t}{\Delta x}}\left[f(u_{{i+1/2}}^{{n+1/2}})-f(u_{{i-1/2}}^{{n+1/2}})\right] $$

In [22]:
def LW_cons(p):    
    """ Lax-Wendroff conservative scheme. """
    u_right = 0.5*(np.roll(p, -1) + p) - dt/(2*dx)*(flux(1, p) - flux(0, p))
    u_left  = 0.5*(p + np.roll(p, 1))  - dt/(2*dx)*(flux(0, p) - flux(-1, p))

    f_right = v0*(1 - u_right/rho0)*u_right
    f_left = v0*(1 - u_left/rho0)*u_left
    
    p += -dt/dx*(f_right - f_left)
    return p

anim = animation.FuncAnimation(fig, animate, init_func=init, fargs=(LW_cons,),
                               frames=int(T/dt), interval=50, blit=True)
p = np.copy(p0)
ax.set_title("Lax-Wendroff Conservative")
rc('animation', html='html5')
HTML(anim.to_html5_video())
anim


### Lax-Freidrich Finite Volumes Scheme

 $$ u_i^{n+1} = u^n_i - \frac{\Delta t}{ \Delta x} \left( \hat{f}^n_{i+1/2} - \hat{f}^n_{i-1/2} \right) $$
 
$$ {\displaystyle {\hat {f}}_{i-1/2}^{n}={\frac {1}{2}}\left(f_{i-1}+f_{i}\right)-{\frac {\Delta x}{2\Delta t}}\left(u_{i}^{n}-u_{i-1}^{n}\right).}  \hat{f}^n_{i-1/2} = \frac{1}{2} \left( f_{i-1} + f_{i} \right) - \frac{ \Delta x}{ 2 \Delta t  } \left( u^n_{i} - u^n_{i-1} \right) $$ 

In [ ]:
def LFFV(Nx=Nx,Nt=Nt,dx=dx,dt=dt,L=L,T=T,p0=p0):
    P = np.copy(p0)
    S = np.zeros((Nx,Nt))
    
    for i in np.arange(0,T,dt):
        f = (1.-P)*P
        P += -(dt/dx)*((0.5*(np.roll(f,-1)+f)-(dt/(2*dx))*(np.roll(P,-1)-P))-\\
                       (0.5*(np.roll(f,1)+f)-(dt/(2*dx))*(P-np.roll(P,1)))) 
        S[int(i/dt),:] = P
        line, = plt.plot(P)
        plt.draw()
        plt.title('Time = ' + str(i) + ' Total Cars = ' + str(np.sum(P)))
        plt.pause(0.01)
        line.remove()
    plt.close()
    return S

## Gudonov Method

## An Exact Numerical Solution by The Lagrange Method 

This method uses the total derivative of each point (points being sampled from the initial conditions) to calculate how much each point moves in space per unit time.

In dt time, each point moves a distance = vdt where velocity for WLR equation = $1-2\rho$. This perfectly works up until the shock point an dit exact but after the schock it becomes unphysical.

In [ ]:
def Lagrange(Nx=Nx,Nt=Nt,dx=dx,dt=dt,L=L,T=T,p0=p0):
    #initial conditions:
    #Sin Wave
    p0 = 0.2 + 0.1*(np.sin(np.arange(0,2*np.pi,2*np.pi/Nx)))
    x0 = np.arange(0,2*np.pi,2*np.pi/Nx)
    
    P = np.copy(p0)
    X = np.copy(x0)
    S = np.zeros((Nx,Nt))
    for i in np.arange(0,T,dt):
        X = np.mod(X + (1-2*P)*dt, 2*np.pi)
        S[int(i/dt),:] = X 
        line, = plt.plot(X,P,'ro')
        plt.draw()
        plt.title('Time = ' + str(i) + ' Total Cars = ' + str(np.sum(P)))
        plt.pause(0.01)
        line.remove()
    plt.close()
    return S

# print(Lagrange())